### import libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot
import seaborn

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

### load data

In [3]:
data_raw = pd.read_csv('../data/train.csv', low_memory=False)
data_raw.describe()

,Store,DayOfWeek,Sales,Customers,Open,Promo,SchoolHoliday
count,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06,1.017209e+06
mean,5.584297e+02,3.998341e+00,5.773819e+03,6.331459e+02,8.301067e-01,3.815145e-01,1.786467e-01
std,3.219087e+02,1.997391e+00,3.849926e+03,4.644117e+02,3.755392e-01,4.857586e-01,3.830564e-01
min,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.800000e+02,2.000000e+00,3.727000e+03,4.050000e+02,1.000000e+00,0.000000e+00,0.000000e+00
50%,5.580000e+02,4.000000e+00,5.744000e+03,6.090000e+02,1.000000e+00,0.000000e+00,0.000000e+00
75%,8.380000e+02,6.000000e+00,7.856000e+03,8.370000e+02,1.000000e+00,1.000000e+00,0.000000e+00
max,1.115000e+03,7.000000e+00,4.155100e+04,7.388000e+03,1.000000e+00,1.000000e+00,1.000000e+00


In [4]:
store_raw = pd.read_csv('../data/store.csv', low_memory=False)
store_raw.describe()

,Store,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear
count,1115.00000,1112.000000,761.000000,761.000000,1115.000000,571.000000,571.000000
mean,558.00000,5404.901079,7.224704,2008.668857,0.512108,23.595447,2011.763573
std,322.01708,7663.174720,3.212348,6.195983,0.500078,14.141984,1.674935
min,1.00000,20.000000,1.000000,1900.000000,0.000000,1.000000,2009.000000
25%,279.50000,717.500000,4.000000,2006.000000,0.000000,13.000000,2011.000000
50%,558.00000,2325.000000,8.000000,2010.000000,1.000000,22.000000,2012.000000
75%,836.50000,6882.500000,10.000000,2013.000000,1.000000,37.000000,2013.000000
max,1115.00000,75860.000000,12.000000,2015.000000,1.000000,50.000000,2015.000000


In [5]:
test_raw = pd.read_csv('../data/test.csv', low_memory=False)
test_raw.describe()

,Id,Store,DayOfWeek,Open,Promo,SchoolHoliday
count,41088.000000,41088.000000,41088.000000,41077.000000,41088.000000,41088.000000
mean,20544.500000,555.899533,3.979167,0.854322,0.395833,0.443487
std,11861.228267,320.274496,2.015481,0.352787,0.489035,0.496802
min,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,10272.750000,279.750000,2.000000,1.000000,0.000000,0.000000
50%,20544.500000,553.500000,4.000000,1.000000,0.000000,0.000000
75%,30816.250000,832.250000,6.000000,1.000000,1.000000,1.000000
max,41088.000000,1115.000000,7.000000,1.000000,1.000000,1.000000


### exploration starts

#### Missing data

In [6]:
def display_missing(name, data):
    report = pd.DataFrame(columns=['column', 'missing', 'total'])
    for column in data.columns:
        null_col = pd.isna(data[column])
        if null_col.sum() > 0:
            report.loc[-1] = [column, null_col.value_counts()[True], data.shape[0]]
            report.index = report.index + 1
    display(name, report.sort_index())


In [7]:
display_missing('train data', data_raw)
display_missing('store data', store_raw)
display_missing('test data', test_raw)

'train data'

,column,missing,total


'store data'

,column,missing,total
0,PromoInterval,544,1115
1,Promo2SinceYear,544,1115
2,Promo2SinceWeek,544,1115
3,CompetitionOpenSinceYear,354,1115
4,CompetitionOpenSinceMonth,354,1115
5,CompetitionDistance,3,1115


'test data'

,column,missing,total
0,Open,11,41088


#### Cardinality

In [8]:
def display_cadinality(name, data):
    report = pd.DataFrame(columns=['column', 'cardinality', 'count'])
    for column in data.columns:
        report.loc[-1] = [column, len(data[column].unique()), data.shape[0]]
        report.index = report.index + 1
    display(name, report.sort_index())

In [9]:
display_cadinality('train data', data_raw)
display_cadinality('store data', store_raw)
display_cadinality('test data', test_raw)

'train data'

,column,cardinality,count
0,SchoolHoliday,2,1017209
1,StateHoliday,4,1017209
2,Promo,2,1017209
3,Open,2,1017209
4,Customers,4086,1017209
5,Sales,21734,1017209
6,Date,942,1017209
7,DayOfWeek,7,1017209
8,Store,1115,1017209


'store data'

,column,cardinality,count
0,PromoInterval,4,1115
1,Promo2SinceYear,8,1115
2,Promo2SinceWeek,25,1115
3,Promo2,2,1115
4,CompetitionOpenSinceYear,24,1115
5,CompetitionOpenSinceMonth,13,1115
6,CompetitionDistance,655,1115
7,Assortment,3,1115
8,StoreType,4,1115
9,Store,1115,1115


'test data'

,column,cardinality,count
0,SchoolHoliday,2,41088
1,StateHoliday,2,41088
2,Promo,2,41088
3,Open,3,41088
4,Date,48,41088
5,DayOfWeek,7,41088
6,Store,856,41088
7,Id,41088,41088
